In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import warnings
warnings.filterwarnings('ignore')
import joblib
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
data=pd.read_pickle('data.pkl')
valid=data[(data['d']>=1914)&(data['d']<1942)][['id','d','sold']]
test=data[data['d']>=1942][['id','d','sold']]

In [3]:
def SVR_model(X_train,y_train,X_valid,y_valid):
    model = SVR(kernel = 'rbf')
    model.fit(X_train,y_train)
    y_pred = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_pred)
    print ("RMSE:",mse**(1/2))

    return model

In [4]:
cal=pd.read_csv('./data/calendar.csv')
sales_train_evaluation=pd.read_csv('data/sales_train_evaluation.csv')

In [5]:
def downcase(df):  
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

In [6]:
cal=downcase(cal)
sales_train_evaluation=downcase(sales_train_evaluation)
catalog_all=pd.melt(sales_train_evaluation,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='sold')
catalog_all=pd.merge(catalog_all,cal,on='d',how='left')
d_id=dict(zip(catalog_all['id'].cat.codes,catalog_all['id']))
del cal,sales_train_evaluation
gc.collect

<function gc.collect(generation=2)>

In [9]:
for i in range(10):
    df= data[data['store_id']==i]
    X_train,y_train=df[df['d']<1914].drop('sold',axis=1),df[df['d']<1914]['sold']
    X_valid,y_valid=df[(df['d']>=1914)&(df['d']<1942)].drop('sold',axis=1),df[(df['d']>=1914)&(df['d']<1942)]['sold']
    X_test= df[df['d']>=1942].drop('sold',axis=1)
    print (f'-----Train model for store {i}------')
    model= SVR_model(X_train,y_train,X_valid,y_valid)
    
    print (f'-----predicting for store {i}------')
    pred_val= model.predict(X_valid)
    valid.loc[X_valid.index,'sold']=pred_val
    pred_eva=model.predict(X_test) 
    test.loc[X_test.index,'sold']= pred_eva
    print ('----------store model---------------')
    filename=f'model_store_SVR_{i}.pkl'
    joblib.dump(model,filename)
    del model,X_train,y_train,X_valid,y_valid,X_test
    gc.collect()

-----Train model for store 0------
